In [1]:
import sys
import os
from pymongo.mongo_client import MongoClient

In [2]:
config_path = os.getcwd().replace('\\scripts','')
sys.path.insert(0, config_path)

from config import settings 

In [3]:
def generate_kept_fields(arr):
    kept_fields = []
    for item in arr:
        if item not in settings.excluded_commodities:
            kept_fields.append(item)
    return kept_fields

In [4]:
generate_kept_fields([item for item in MongoClient(settings.mongo_client)[settings.mongo_default_db].list_collection_names()])

['BROCCOLI',
 'SELFPROPELLED',
 'INTERNET',
 'SHORTTERMWOODYCROPS',
 'GRAPES',
 'CAULIFLOWER',
 'TAXES',
 'CREAM',
 'DECIDUOUSSHRUBS',
 'TALLOW',
 'MOLLUSKS',
 'OIL-BEARINGCROPS',
 'POTATOES',
 'SUPPLIES',
 'KIWIFRUIT',
 'CHEESE',
 'CRUDEPINEGUM',
 'REPAIRS',
 'RENT',
 'BUCKWHEAT',
 'PEARS',
 'SUPPLIESREPAIRS',
 'PITW',
 'WATERCRESS',
 'WOOL',
 'PEAS',
 'LEMONSLIMES',
 'TANGERINES',
 'VEGETABLESMIXED',
 'FOODGRAINS',
 'OLIVES',
 'FERTILIZERMIXED',
 'LANDAREA',
 'MINK',
 'RICE',
 'CUCUMBERS',
 'LONGAN',
 'CHERRIES',
 'BLUEBERRIES',
 'CUTCULTIVATEDGREENS',
 'CANOLA',
 'SHEEP',
 'BUTTERMILK',
 'TRACTORS',
 'BRUSSELSSPROUTS',
 'KUMQUATS',
 'GUAR',
 'FEEDGRAINSHAY',
 'TURNIPS',
 'CELERY',
 'MILK',
 'PISTACHIOS',
 'BEETS',
 'FOODFISH',
 'CABBAGE',
 'COFFEE',
 'RHEAS',
 'PINEAPPLES',
 'COTTON',
 'NONFARMSECTOR',
 'CRANBERRIES',
 'FIELDWORK',
 'WALNUTS',
 'WATERICES',
 'MACADAMIAS',
 'STRAWBERRIES',
 'TANGELOS',
 'SOIL',
 'CORN',
 'PECANS',
 'LOGANBERRIES',
 'HOPS',
 'FUELS',
 'OIL',
 'YOGURT'